# Correr experimentos
En este archivo está el código para correr los experimentos y escribir los resultados en archivos CSV.
> Los archivos se guardan en la carpeta _resultados_.

In [4]:
import math, subprocess
import pandas as pd
import numpy as np
from IPython.display import display, clear_output

A continuación leemos los datasets en dataframes de Pandas.

In [5]:
def leer_instancia(path_instancia):
    with open(path_instancia, "r") as f:
        return f.read();

In [8]:
df_mejor_caso_bt_f = pd.read_csv("instancias/mejor-caso-bt-f/indice.csv");


In [11]:
df_dinamica = pd.read_csv("instancias/dinamica/indice.csv");

In [80]:
df_bt= pd.read_csv("instancias/bt/indice.csv");

In [6]:
df_fbms = pd.read_csv("instancias/muchas-soluciones-validas/indice.csv");
df_fbps = pd.read_csv("instancias/pocas-soluciones-validas/indice.csv");

In [14]:
df_fb2 = pd.read_csv("instancias/fb2/indice.csv");

La siguiente función sirve para correr el código sobre una instancia ejecutando un método en particular.
- FB: Fuerza bruta
- BT: Backtracking con ambas podas.
- BT-F: Backtracking solamente con poda por factibilidad.
- BT-O: Backtracking solamente con poda por optimalidad.
- PD: Programación dinámica.

In [7]:
def correr_experimento(metodo, archivo_instancia):
    # Leer archivo de la instancia.
    instancia = leer_instancia(archivo_instancia)
    
    # Crear proceso para ejecutar el codigo.
    process = subprocess.Popen(["../main", metodo], stderr=subprocess.PIPE, stdout=subprocess.PIPE, stdin=subprocess.PIPE, universal_newlines = True)

    # Poner la instancia en la entrada estandar.
    process.stdin.write(instancia)
    process.stdin.flush()

    # Correr experimento.
    exit_code = process.wait()

    # Verificar que el proceso no fallo.
    if exit_code != 0: raise(F"Hubo un error en la experimentacion para el algoritmo: {algoritmo} con la instancia {archivo_instancia}.")
    # Leer salida de STDERR con los tiempos de ejecucion de cada metodo.
    tiempo_de_ejecucion = float(process.stderr.read());
    
    process.stdin.close();
    process.stdout.close();
    process.stderr.close();
    
    return tiempo_de_ejecucion;

## Corremos los experimentos
Vamos a guardar una tabla con las ejecuciones y sus respectivos tiempos.

In [8]:
experimentos = [];

## Experimento Fuerza Bruta : R variable para distintos ns fijos

In [36]:
for i in range(0, df_fb2.shape[0]):
    fila = df_fb2.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Experimento Fuerza Bruta : Pocas/Muchas soluciones

In [22]:
for i in range(0, df_fbms.shape[0]):
    fila = df_fbms.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);
    
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "FB", fila["archivo"]]);

## Experimento Back Tracking - Pocas/Muchas soluciones (Comparacion con FB)

In [9]:
for i in range(0, df_fbms.shape[0]):
    fila = df_fbms.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);
    
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);

## Experimento Back Tracking - Todos Rompen
Correr BT-F para el caso en que todos los productos rompen el jambotubo.

In [42]:
for n in range(0, df_mejor_caso_bt_f.shape[0]):
    fila_n = df_mejor_caso_bt_f.iloc[n];
    experimentos.append([fila_n["dataset"], n, fila_n["R"], "BT-F", fila_n["archivo"]]);

## Experimento Campana de Back Tracking

In [84]:
for i in range(0, df_bt.shape[0]):
    fila = df_bt.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);

## Experimento Programacion Dinamica


In [55]:
for i in range(0, df_dinamica.shape[0]):
    fila = df_dinamica.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);  

## Experimento 1 :BT vs. PD

In [1]:
Rs_max = [235,223,79,169,47,153,51,15,19,119,27,29,35,43,55,73,65,73,77,93,87,95,107,119,119,145,169,151,183,191]

In [99]:
for i in range(0, df_bt.shape[0]):
    fila = df_bt.iloc[i];
    n = fila["n"] 
    R = fila["R"]
    if (Rs_max[n - 2]  == R):
        experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);

## Experimento 2 :BT vs. PD

In [107]:
for i in range(0, df_fbps.shape[0]):
    fila = df_fbps.iloc[i];
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "PD", fila["archivo"]]);
    experimentos.append([fila["dataset"], fila["n"], fila["R"], "BT", fila["archivo"]]);    

## Ejecutar los experimentos y guardar los resultados en un archivo CSV.
Este paso puede tardar unos minutos hasta terminar de ejecutarse.

In [23]:
columnas = ["dataset", "n", "R", "metodo", "tiempo"];
filas = [];
numero = 1
T = 5 # Numero de veces que se ejecuta cada experimento (para mayor fidelidad del tiempo).
for experimento in experimentos:
    # Voy mostrando que experimento se esta ejecutando.
    clear_output(wait=True)
    display('Experimento: ' + str(numero) + "/" + str(len(experimentos)))
    numero += 1
    
    # Ejecutamos el experimento T veces y obtenemos la mediana.
    tiempos = []
    resultados = []
    for i in range(0, T):
        t = correr_experimento(experimento[3], experimento[4])
        #t = exp[0]
        #res = exp[1]
        #resultados.append(res)
        tiempos.append(t);
    tiempo = np.median(tiempos);
    #print(resultados)
    filas.append([experimento[0], experimento[1], experimento[2], experimento[3], tiempo]);


'Experimento: 96/96'

In [24]:
df_resultado = pd.DataFrame(filas, columns=columnas);
df_resultado

,dataset,n,R,metodo,tiempo
0,muchas-soluciones-validas,1,1000,BT,0.001705
1,muchas-soluciones-validas,2,1000,BT,0.001506
2,muchas-soluciones-validas,3,1000,BT,0.001881
3,muchas-soluciones-validas,4,1000,BT,0.001656
4,muchas-soluciones-validas,5,1000,BT,0.001649
...,...,...,...,...,...
91,pocas-soluciones-validas,20,1000,FB,7.532530
92,pocas-soluciones-validas,21,1000,FB,15.133200
93,pocas-soluciones-validas,22,1000,FB,29.921700
94,pocas-soluciones-validas,23,1000,FB,59.889800


In [27]:
df_aguardar = pd.read_csv("resultados/resultado.csv");
df_aguardar 

,dataset,n,R,metodo,tiempo


In [111]:
df_aguardar["dataset"].unique()

array(['fb2', 'muchas-soluciones-validas', 'pocas-soluciones-validas',
       'mejor-caso-bt-f', 'dinamica', 'bt'], dtype=object)

In [28]:
df_aguardar = df_aguardar.append(df_resultado,ignore_index = True)
df_aguardar

,dataset,n,R,metodo,tiempo
0,muchas-soluciones-validas,1,1000,BT,0.001705
1,muchas-soluciones-validas,2,1000,BT,0.001506
2,muchas-soluciones-validas,3,1000,BT,0.001881
3,muchas-soluciones-validas,4,1000,BT,0.001656
4,muchas-soluciones-validas,5,1000,BT,0.001649
...,...,...,...,...,...
91,pocas-soluciones-validas,20,1000,FB,7.532530
92,pocas-soluciones-validas,21,1000,FB,15.133200
93,pocas-soluciones-validas,22,1000,FB,29.921700
94,pocas-soluciones-validas,23,1000,FB,59.889800


In [29]:
df_aguardar.to_csv("resultados/resultado.csv", index=False, header=True);